In [1]:
import gzip, pickle, pathlib, torch
data_path = pathlib.Path("/content/mnist.pkl.gz")

def get_data(data_path):
    with gzip.open(data_path) as f:
        (x_train, y_train), (x_valid, y_valid), (x_test, y_test)= pickle.load(f, encoding='latin-1')
        return map(torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test))

x_train, y_train, x_valid, y_valid, x_test, y_test = get_data(data_path)

In [2]:
class Dataset():
    def __init__(self, x, y): self.x,self.y = x,y
    def __len__(self): return len(self.x)
    def __getitem__(self, i): return self.x[i],self.y[i]

In [3]:
train_ds, valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

In [7]:
# loss, number of nodes, bachsize configures

nh, bs = 50, 512
c = y_train.max().item()+1

from torch.functional import F

loss_func = F.cross_entropy

nh, bs, loss_func, c

(50, 512, <function torch.nn.functional.cross_entropy>, 10)

In [10]:
#  get data

from torch.utils.data import DataLoader

def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(train_ds, batch_size=bs, shuffle=True, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, **kwargs))
    
class DataBunch():
    def __init__(self, train_dl, valid_dl, c=None):
        self.train_dl,self.valid_dl,self.c = train_dl,valid_dl,c

    @property
    def train_ds(self): return self.train_dl.dataset

    @property
    def valid_ds(self): return self.valid_dl.dataset

data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [11]:
def test(name): return name

In [18]:
test(None or 'var')

'var'

In [17]:
test('None' or 'var')

'None'

In [19]:
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')
def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

# set callback function
class Callback():
    _order = 0
    def set_runner(self, run): self.run = run
    def __getattr__(self, k): return getattr(self.run, k)

    @property
    def name(self):
        name = re.sub(r"Callback$", "", self.__class__.__name__)
        # you can get 'callback' as default name.. wow
        return calmel2snake(name or 'callback')
    
    def __call__(self, cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): return True
        # return False if callback doesn't have 'cb' (i.e. class itself) or cb_name (i.e. callback attr).
        return False

In [20]:
class TrainEvalCallback(Callback):
    pass

```python
from functools import partial

from torch import nn, optim, tensor


def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()

def get_model(data, lr=0.5, nh=50):
    m = data.train_ds.x.shape[1]
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,data.c))
    return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model,self.opt,self.loss_func,self.data = model,opt,loss_func,data
```        